In [ ]:
!gsutil cp -r gs://cloud-ai-platform-c541b3e3-934f-414e-9196-8e2bf7a7fb59 .

In [3]:
from google.cloud import aiplatform


def export_model_sample(
    project: str,
    model_id: str,
    gcs_destination_output_uri_prefix: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
    timeout: int = 300,
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.ModelServiceClient(client_options=client_options)
    output_config = {
        "artifact_destination": {
            "output_uri_prefix": gcs_destination_output_uri_prefix
        },
        # For information about export formats: https://cloud.google.com/ai-platform-unified/docs/export/export-edge-model#aiplatform_export_model_sample-drest
        "export_format_id": "tf-saved-model",
    }
    name = client.model_path(project=project, location=location, model=model_id)
    response = client.export_model(name=name, output_config=output_config)
    print("Long running operation:", response.operation.name)
    print("output_info:", response.metadata.output_info)
    export_model_response = response.result(timeout=timeout)
    print("export_model_response:", export_model_response)

export_model_sample(
    project="llm-db",
    model_id="6131366119153336320",
    gcs_destination_output_uri_prefix="gs://cloud-ai-platform-c541b3e3-934f-414e-9196-8e2bf7a7fb59/exported_model",
)


Long running operation: projects/153731718742/locations/us-central1/models/6131366119153336320/operations/4738438096834527232
output_info: artifact_output_uri: "gs://cloud-ai-platform-c541b3e3-934f-414e-9196-8e2bf7a7fb59/exported_model/model-6131366119153336320/tf-saved-model/2025-05-15T05:42:28.709186Z"

export_model_response: 


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio   # must come before loading the model

# path to the directory containing saved_model.pb
MODEL_DIR = "cloud-ai-platform-c541b3e3-934f-414e-9196-8e2bf7a7fb59/model-6131366119153336320/tf-saved-model/2025-05-15T03:23:14.175361Z/predict/001"

# load the model
loaded = tf.saved_model.load(MODEL_DIR)
infer = loaded.signatures["serving_default"]

# read your test set
df_test = pd.read_csv("cloud-ai-platform-c541b3e3-934f-414e-9196-8e2bf7a7fb59/hf_models_withmodelcard_nov2024.csv")
features = ["task_group","author_category","language_category","location"]  # whatever your model signature expects

# build inputs dict
inputs = {}
for col in features:
    # if your model expects string tensors:
    inputs[col] = tf.constant(df_test[col].astype(str).values)
    # or float: tf.constant(df_test[col].values, dtype=tf.float32)

# call the model
outputs = infer(**inputs)

# inspect output keys
print("Available outputs:", list(outputs.keys()))
# typically something like 'output_0' or 'predictions'
pred_tensor = outputs["output_0"]    # replace with the right key

# if it’s logits or probabilities, you might need argmax
if pred_tensor.shape[-1] > 1:
    pred_indices = tf.argmax(pred_tensor, axis=-1).numpy()
else:
    # if binary, maybe threshold
    pred_indices = (pred_tensor.numpy() > 0.5).astype(int).flatten()

# decode indices back to category names if you kept a LabelEncoder
# (assuming you saved it when exporting)
#    from sklearn.preprocessing import LabelEncoder
#    le = load_my_label_encoder()
#    preds = le.inverse_transform(pred_indices)

# attach predictions
df_test["pred_idx"] = pred_indices
print(df_test.head())


2025-05-14 22:00:31.913000: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747285231.934045   19282 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747285231.940334   19282 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747285231.956565   19282 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747285231.956582   19282 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747285231.956584   19282 computation_placer.cc:177] computation placer alr

RuntimeError: Op type not registered 'DecodeProtoSparseV4' in binary running on cadejo.cs.usfca.edu. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib (e.g. `tf.contrib.resampler`), accessing should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.

In [1]:
import tensorflow_io as tfio   # registers DecodeProtoSparseV4 and friends
import tensorflow as tf
import pandas as pd

# 1) point this at the directory that contains saved_model.pb and variables/
MODEL_DIR = "cloud-ai-platform-c541b3e3-934f-414e-9196-8e2bf7a7fb59/model-6131366119153336320/tf-saved-model/2025-05-15T03:23:14.175361Z/predict/001"

2025-05-14 21:11:54.372504: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747282314.393301   18855 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747282314.400085   18855 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747282314.416272   18855 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747282314.416289   18855 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747282314.416291   18855 computation_placer.cc:177] computation placer alr

In [2]:
# 2) load your own data
df = pd.read_csv("cloud-ai-platform-c541b3e3-934f-414e-9196-8e2bf7a7fb59/hf_models_withmodelcard_nov2024.csv")

print(df.downloads_category.value_counts())
# keep only the feature columns your model expects
feature_columns = ["task_group", "author_category", "language_category", "location", "downloads_category"]
X = df[feature_columns]

downloads_category
Very Low    807979
Low         249325
Mid          26888
High         14068
Name: count, dtype: int64


In [3]:
# 3) load the SavedModel
#    this gives you an object whose `signatures` dict contains a 'serving_default' fn
loaded = tf.saved_model.load(MODEL_DIR)
infer = loaded.signatures["serving_default"]

I0000 00:00:1747282327.253322   18855 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15657 MB memory:  -> device: 0, name: Quadro P5000, pci bus id: 0000:17:00.0, compute capability: 6.1
I0000 00:00:1747282327.253990   18855 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 15657 MB memory:  -> device: 1, name: Quadro P5000, pci bus id: 0000:65:00.0, compute capability: 6.1


RuntimeError: Op type not registered 'DecodeProtoSparseV4' in binary running on cadejo.cs.usfca.edu. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib (e.g. `tf.contrib.resampler`), accessing should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.

In [ ]:
# 4) build a dict of tf.constant inputs
#    keys must exactly match the input names in your model signature (you can inspect infer.structured_input_signature)
model_inputs = {}
for col in feature_columns:
    # if your model expects strings for categorical features:
    model_inputs[col] = tf.constant(X[col].astype(str).values)
    # if it expects numeric, you could use tf.constant(X[col].values, dtype=tf.float32)

In [ ]:
# 5) call the model
outputs = infer(**model_inputs)

# 6) extract predictions
#    the key here (“predictions” or “outputs”) depends on your model’s signature
pred = outputs["predictions"].numpy()  
# or if it’s a different key:
# pred = outputs["logistic_regression_prediction"].numpy()

In [ ]:
# 7) attach back to your DataFrame
df["predicted_downloads_category"] = pred

print(df[["downloads_category", "predicted_downloads_category"]].head())